# Landsat Collection 2 on AWS

This notebook explores the Landsat Collection 2 data on AWS:

- [Landsat Collection 2 STAC API](https://landsatlook.usgs.gov/stac-server), a catalog of Landsat data
- [pystac-client](https://pystac-client.readthedocs.io/) for searching and access data
- [OpenDataCube](https://www.opendatacube.org/) and [odc-stac](https://odc-stac.readthedocs.io/) for loading STAC assets and representing geospatial data as XArrays
- [XArray](http://xarray.pydata.org/en/stable/), [pandas](https://pandas.pydata.org/) and [geopandas](https://geopandas.org/) for manipulating data
- [Dask](https://dask.org/) for performing parallel, distributed computing
- [Coiled.io](https://coiled.io/), a service for hosting Dask clusters
- [hvplot](https://hvplot.holoviz.org/) for visualization

Shown will be how to find data for an area of interest, explore the resulting metadata, perform calculations, and visualize the results.

Created by [Element 84](http://element84.com/)

In [ ]:
# initial imports and reusable functions

import holoviews as hv
hv.extension('bokeh')

from copy import deepcopy
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
from shapely.geometry import shape
import os
os.environ['AWS_REQUEST_PAYER'] = 'requester'

In [ ]:
# create a function for later reuse
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=600, frame_height=600,
                             line_width=3, **kwargs)

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

# Search for data

Use pystac-client to find data in the Landsat STAC API. First, print up a table of all the STAC Collections in the API.

In [ ]:
# Open the Landsat STAC API

from pystac_client import Client
URL = 'https://landsatlook.usgs.gov/stac-server'
cat = Client.open(URL)
print(cat)

collections = [(c.id, c.title) for c in cat.get_collections()]
pd.set_option("display.max_colwidth", 150)
df = pd.DataFrame(collections, columns=['id', 'title'])
df

Fetch the collection of interest: Landsat Collection 2, Level 2 Surface Reflectance (landsat-c2l2-sr) and print the assets that are available.

In [ ]:
collection_id = 'landsat-c2l2-sr'

collection = cat.get_collection(collection_id)
pd.DataFrame.from_dict(collection.to_dict()['item_assets'], orient='index')

Change the AOI, search parameters here, and print how many matching scenes there are.

In [ ]:
import geopandas as gpd
import json

aoi = gpd.read_file('../aois/bear-fire.geojson')
geom = json.loads(aoi['geometry'].to_json())['features'][0]['geometry']

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    collections = [collection_id],
    intersects = aoi['geometry'][0],
    datetime = "2021-01-01/2021-03-31",
    query = ["eo:cloud_cover<10"],
    limit = 100
)

print(f"{search.matched()} items found")

# Use GeoPandas to view footprints

The cell below fetches all the STAC Items and updates the URLs to use the provided s3 URLs which can be used for direct access rather than the default https URLs. This is because the alternate extension is not yet supported in PySTAC, when it is there will be an easier way to specify which alternate URL, if any, to use for the assets.

Then, we create a GeoDataFrame for visualizing the footprints.

In [ ]:
# Get all items as a dictionary
items_dict = search.get_all_items_as_dict()['features']

# update URLs to use s3
for item in items_dict:
    for a in item['assets']:
        if 'alternate' in item['assets'][a] and 's3' in item['assets'][a]['alternate']:
            item['assets'][a]['href'] = item['assets'][a]['alternate']['s3']['href']
        item['assets'][a]['href'] = item['assets'][a]['href'].replace('usgs-landsat-ard', 'usgs-landsat')

# Create GeoDataFrame from Items
items_gdf = items_to_geodataframe(items_dict)

print(f"{len(items_dict)} items found")

pd.reset_option("display.max_colwidth")
items_gdf.head()

In [ ]:
plot_polygons(aoi) * items_gdf.hvplot.paths(geo=True)

# OpenDataCube

Now we'll turn the set of scenes into a virtual datacube. None of the data will actually be read yet.

The configuration string (`cfg`) is for providing additional info not currently available in the STAC Items, but will be in the future.

In [ ]:
import yaml

cfg = """---
landat-c2l2-sr:
  measurements:
    '*':
      dtype: uint16
      nodata: 0
      unit: 'm'
"""
cfg = yaml.load(cfg, Loader=yaml.CSafeLoader)

Here we load as a DataCube. A PySTAC ItemCollection is created from the found STAC Items, and we specify various parameters, such as bands of interest and chunk size.

In [ ]:
%%time

from odc.stac import stac_load

# Create PySTAC ItemCollection
item_collection = pystac.ItemCollection(items_dict)

# default to CRS and resolution from first Item
from pystac.extensions.projection import ProjectionExtension
from pyproj import CRS

proj = ProjectionExtension.ext(item_collection[0])
output_crs = CRS.from_epsg(proj.epsg)
resolution = (proj.transform[4], proj.transform[0])

dc = stac_load(item_collection,
               bands=['red', 'blue', 'green', 'nir08'],
               chunks={"x": 2048, "y": 2048},
               output_crs=output_crs,
               resolution=resolution,
               groupby='solar_day',
               stac_cfg=cfg
)
dc

# Calculations

The datacube currently contains complete Items, we want to clip these to our geometry of interest. We will then also create an RGB datacube representation, and generate an NDVI datacube.

In [ ]:
%%time

import rioxarray

dc = dc.rio.clip([geom], crs='epsg:4326')
dc

In [ ]:
from odc.algo import to_rgba

vis = to_rgba(dc, clamp=(1, 20000), bands=['red', 'green', 'blue'])
vis

In [ ]:
ndvi = ((dc['nir08'] - dc['red']) / (dc['nir08'] + dc['red'])).clip(0, 1)
ndvi.name = 'ndvi'
ndvi

# Start Dask Client

Start either a local Dask, or use [coiled.io](coiled.io)

In [ ]:
# local Dask

from dask.distributed import Client
client = Client()
client

# Compute

Now, we kick off our Dask computation by using the Dask persist function, which will keep the data in memory on the cluster for faster access later.

The Dask `compute` function is used when we actually want the data, such as displaying it.

In [ ]:
client

In [ ]:
%%time
from dask.distributed import wait

vis = client.persist(vis)
_ = wait(vis)

In [ ]:
%%time
vis_ = vis.compute()
vis_.plot.imshow(col='time', rgb='band', col_wrap=5, robust=True)

In [ ]:
import hvplot.xarray
import panel as pn

In [ ]:
vis_.spatial_ref.crs_wkt

In [ ]:
hvplot_kwargs = dict(crs=32610, tiles='OSM', 
                frame_width=400, widgets={'time': pn.widgets.Select})

vis_.hvplot.rgb(x='x', y='y', bands='band', **hvplot_kwargs)

In [ ]:
vis_.hvplot.rgb(x='x', y='y', bands='band', **hvplot_kwargs)

In [ ]:
%%time
ndvi_ = ndvi.compute()
ndvi_.hvplot(x='x', y='y', **hvplot_kwargs)

Create an animated GIF of NDVI over time using `geogif` with the fetched results.

In [ ]:
from geogif import gif

#ndvi_ = ndvi_c.transpose('time','x','y').compute()

gif(ndvi_, fps=1, cmap='YlGn')

In [ ]:
%%time
ndvi_mean = ndvi.mean(dim=['x', 'y']).compute()
ndvi_mean.hvplot()

# Shutdown cluster

Shut down the cluster.

In [ ]:
client.close()